# Выбор локации для скважины

В добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой Bootstrap.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

<div class="alert alert-info">
Загрузим и подготовим данные</div

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression
import random
from numpy.random import RandomState
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import (
    confusion_matrix, 
    f1_score, 
    mean_squared_error, 
    roc_auc_score,
    roc_curve
)
import warnings
warnings.filterwarnings('ignore')


In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [7]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


<div class="alert alert-info">
Пропусков и некорректных значений нет, типа данных тоже нас устраивают</div

## Обучение и проверка модели

<div class="alert alert-info">
Обучим и проверим модель для каждого региона. Разобьём данные на обучающую и валидационную выборки в соотношении 75:25.
</div

In [8]:
data_0_train, data_0_valid = train_test_split(data_0, test_size=0.25, random_state=12345)
data_0_train.shape

(75000, 5)

In [9]:
features_0_train = data_0_train.drop(['id', 'product'], axis=1)
target_0_train = data_0_train['product']

features_0_valid = data_0_valid.drop(['id', 'product'], axis=1)
target_0_valid = data_0_valid['product']

features_0_train.head()

,f0,f1,f2
27212,0.022450,0.951034,2.197333
7866,1.766731,0.007835,6.436602
62041,0.724514,0.666063,1.840177
70185,-1.104181,0.255268,2.026156
82230,-0.635263,0.747990,6.643327


In [10]:
data_1_train, data_1_valid = train_test_split(data_1, test_size=0.25, random_state=12345)
data_1_train.shape

(75000, 5)

In [11]:
features_1_train = data_1_train.drop(['id', 'product'], axis=1)
target_1_train = data_1_train['product']

features_1_valid = data_1_valid.drop(['id', 'product'], axis=1)
target_1_valid = data_1_valid['product']

In [12]:
data_2_train, data_2_valid = train_test_split(data_2, test_size=0.25, random_state=12345)
data_2_train.shape

(75000, 5)

In [13]:
features_2_train = data_2_train.drop(['id', 'product'], axis=1)
target_2_train = data_2_train['product']

features_2_valid = data_2_valid.drop(['id', 'product'], axis=1)
target_2_valid = data_2_valid['product']

<div class="alert alert-info">
Сохранили предсказания и правильные ответы на валидационной выборке</div

 <div class="alert alert-info">
Обучим модель и сделаем предсказания на валидационной выборке. Так как надо предсказать некий диапазон, а не точное число, нужна задача регрессии, а не классификации</div

<div class="alert alert-info">
Какая метрика подходит для задачи регрессии? Наиболее распространённая метрика качества в задаче регрессии — средняя квадратичная ошибка, MSE </div

In [14]:
model = LinearRegression() 
model.fit(features_0_train, target_0_train) 
predictions_valid_0 = model.predict(features_0_valid) 

result_0 = (mean_squared_error(target_0_valid, predictions_valid_0))**0.5
stock_0 = target_0_valid.mean()
print ("Средний запас предсказанного сырья", stock_0)
print("RMSE модели линейной регрессии на валидационной выборке:", result_0)

Средний запас предсказанного сырья 92.07859674082927
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813


In [15]:
model = LinearRegression() 
model.fit(features_1_train, target_1_train) 
predictions_valid_1 = model.predict(features_1_valid) 

result_1 = (mean_squared_error(target_1_valid, predictions_valid_1))**0.5
stock_1 = target_1_valid.mean()
print ("Средний запас предсказанного сырья", stock_1)
print("RMSE модели линейной регрессии на валидационной выборке:", result_1)

Средний запас предсказанного сырья 68.72313602435997
RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617


In [16]:
model = LinearRegression() 
model.fit(features_2_train, target_2_train) 
predictions_valid_2 = model.predict(features_2_valid) 

result_2 = (mean_squared_error(target_2_valid, predictions_valid_2))**0.5
stock_2 = target_2_valid.mean()
print ("Средний запас предсказанного сырья", stock_2)
print("RMSE модели линейной регрессии на валидационной выборке:", result_2)

Средний запас предсказанного сырья 94.88423280885438
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434


<div class="alert alert-info">
RMSE у двух регионов 37 и 40, а у одного - 0,89. RMSE говорит нам о средней ошибке модели, насколько в среднем наша модель ошибается,  Чем меньше ее значение тем лучше. Видно, что у региона0 и региона2 средний запас сырья больше, чем у региона1. Но среднеквадратичное отклонение больше на два порядка. То есть предположительный запас сырья у региона1 меньше, но предсказать это количество мы можем с большей точностью, чем для других регионов.</div

## Подготовка к расчёту прибыли

<div class="alert alert-info">
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.</div

In [17]:
budget = 10_000_000_000 # бюджет
quantity = 500       # количество исследуемых точек в регионе
quan_best = 200      # количество лучших точек
price = 450_000       # доход с каждой единицы продукта (1000 баррелей сырья)


<div class="alert alert-info">
Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.</div

In [18]:
expense = budget / (quan_best * price)
expense

111.11111111111111

<div class="alert alert-info">
Получилось, что достаточный объём сырья для безубыточной разработки новой скважины превышает средние объемы запасов каждого из регионов. Значит, некоторые скважины убыточные и если мы будем выбирать их в каждом из регионов случайным образом, то с немалой вероятностью можем получить убыток. Нужна модель, которая позволит нам прогнозировать объем продукта в скважине перед началом ее разработки.</div

## Расчёт прибыли и рисков 

<div class="alert alert-info">
Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:</div

<div class="alert alert-info">
Я выбираю случайным образом 500 точек из признаков валидационной выборки, по ним модель делает предсказания, из этих предсказаний выбираю 200 лучших значений. После этого 200 значений суммирую и вычисляю прибыль. 
Подразумевается выбор 200 скважин с максимальными значениями предсказанного объема продукта для каждого региона. Получается мы сортируем предсказания от большего к меньшему и выбираем топ 200 скважин.</div

In [22]:
holes_0 = pd.Series(predictions_valid_0).sort_values(ascending=False).head(200)
holes_0.count()

200

In [23]:
holes_1 = pd.Series(predictions_valid_1).sort_values(ascending=False).head(200)
holes_1.head()

20430    139.818970
7777     139.773423
8755     139.703330
1178     139.560938
4285     139.516754
dtype: float64

In [24]:
holes_2 = pd.Series(predictions_valid_2).sort_values(ascending=False).head(200)
holes_2.head()

22636    165.856833
24690    165.679685
7811     163.439962
1581     162.062589
6751     161.797476
dtype: float64

<div class="alert alert-info">
Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.</div 

In [28]:
holes_0_sum = holes_0.sum()
holes_0_sum

31102.3308388114

In [29]:
holes_1_sum = holes_1.sum()
holes_1_sum

27746.026782163426

In [30]:
holes_2_sum = holes_2.sum()
holes_2_sum

29603.898658318347

<div class="alert alert-info">
Рассчитаем прибыль для полученного объёма сырья.</div 

In [31]:
def profit_gen(holes_sum):
    profit = holes_sum * price - budget
    return profit
profit_gen_0 = profit_gen(holes_0_sum)
profit_gen_1 = profit_gen(holes_1_sum)
profit_gen_2 = profit_gen(holes_2_sum)
print (profit_gen_0)
print (profit_gen_1)
print (profit_gen_2)

3996048877.46513
2485712051.9735413
3321754396.2432556


<div class="alert alert-info">
Посчитаем риски и прибыль для каждого региона:</div 

In [32]:
len(predictions_valid_0)

25000

In [33]:
target_0_valid.count()

25000

In [34]:
predictions_valid_0 = pd.Series(predictions_valid_0)
target_0_valid = target_0_valid.reset_index(drop=True)
predictions_valid_0 = predictions_valid_0.reset_index(drop=True)

target_pred_concat_0 = pd.concat([predictions_valid_0, target_0_valid], axis=1)
target_pred_concat_0 = target_pred_concat_0.rename(columns = {"product":"target", 0: "predictions"})
target_pred_concat_0.tail()

,predictions,target
24995,103.037104,170.116726
24996,85.403255,93.632175
24997,61.509833,127.352259
24998,118.180397,99.782700
24999,118.169392,177.821022


In [35]:
predictions_valid_1 = pd.Series(predictions_valid_1)
target_1_valid = target_1_valid.reset_index(drop=True)
predictions_valid_1 = predictions_valid_1.reset_index(drop=True)

target_pred_concat_1 = pd.concat([predictions_valid_1, target_1_valid], axis=1)
target_pred_concat_1 = target_pred_concat_1.rename(columns = {"product":"target", 0: "predictions"})
target_pred_concat_1.head()

,predictions,target
0,82.663314,80.859783
1,54.431786,53.906522
2,29.748760,30.132364
3,53.552133,53.906522
4,1.243856,0.000000


In [36]:
predictions_valid_2 = pd.Series(predictions_valid_2)
target_2_valid = target_2_valid.reset_index(drop=True)
predictions_valid_2 = predictions_valid_2.reset_index(drop=True)

target_pred_concat_2 = pd.concat([predictions_valid_2, target_2_valid], axis=1)
target_pred_concat_2 = target_pred_concat_2.rename(columns = {"product":"target", 0: "predictions"})
target_pred_concat_2.head()

,predictions,target
0,93.599633,61.212375
1,75.105159,41.850118
2,90.066809,57.776581
3,105.162375,100.053761
4,115.303310,109.897122


<div class="alert alert-info">
Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. </div

In [37]:
state = np.random.RandomState(12345)
target_pred_subsample_0 = target_pred_concat_0.sample(n=500, replace=True, random_state=state)
target_pred_subsample_0

,predictions,target
20962,69.031631,34.280424
11749,108.084038,55.567766
2177,89.065885,26.938283
19876,109.885938,133.142251
11689,71.893169,12.386952
...,...,...
13517,51.883205,90.511196
17127,88.959983,134.992329
341,113.393785,74.874896
3949,54.043185,90.708239


In [38]:
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return (price * selected.sum() - budget)
    
values = []

for i in range(1000):
    target_pred_subsample = target_pred_concat_0.sample(n=500, replace=True, random_state=state)
    values.append(revenue(target_pred_subsample['target'], target_pred_subsample['predictions'], 200))

values = pd.Series(values)
lower_0 = values.quantile(0.025)
upper = values.quantile(0.975) 
mean_0 = values.mean()
risk_0 = (values < 0).mean() * 100
print ("Риски региона_0:", risk_0)
print("Средняя выручка региона_0:", mean_0)
print("2,5%-квантиль:", lower_0)
print (values.head())

Риски региона_0: 6.0
Средняя выручка региона_0: 425368536.2814983
2,5%-квантиль: -102090094.83793654
0    5.363934e+08
1    2.110794e+08
2    2.652803e+08
3    2.719929e+08
4    4.047124e+08
dtype: float64


In [39]:
target_pred_subsample_1 = target_pred_concat_1.sample(n=500, replace=True, random_state=state)
target_pred_subsample_1

,predictions,target
8222,81.097991,80.859783
6498,2.424588,3.179103
16910,2.264602,3.179103
3346,136.767611,137.945408
1041,27.852646,26.953261
...,...,...
12152,30.158440,30.132364
15612,53.339444,53.906522
10011,-0.128454,0.000000
11537,135.196043,134.766305


In [40]:
values = []
for i in range(1000):
    target_pred_subsample = target_pred_concat_1.sample(n=500, replace=True, random_state=state)
    values.append(revenue(target_pred_subsample['target'], target_pred_subsample['predictions'], 200))

values = pd.Series(values)
lower_1 = values.quantile(0.025)
upper = values.quantile(0.975) 
mean_1 = values.mean()
risk_1 = (values < 0).mean() * 100
print ("Риски региона_1:", risk_1)
print("Средняя выручка региона_1:", mean_1)
print("2,5%-квантиль:", lower_1)

Риски региона_1: 0.3
Средняя выручка региона_1: 518749628.38254464
2,5%-квантиль: 129518062.69146843


In [41]:
target_pred_subsample_2 = target_pred_concat_2.sample(n=500, replace=True, random_state=state)
target_pred_subsample_2

,predictions,target
3340,97.754351,69.605753
18089,112.135087,151.982040
22840,74.695896,79.130125
11792,99.762250,52.689146
23329,109.715181,149.943969
...,...,...
14560,101.168118,88.176676
662,85.498749,90.847463
3766,91.956291,56.658706
16046,96.124394,56.101113


In [42]:
values = []
for i in range(1000):
    target_pred_subsample = target_pred_concat_2.sample(n=500, replace=True, random_state=state)
    values.append(revenue(target_pred_subsample['target'], target_pred_subsample['predictions'], 200))

values = pd.Series(values)
lower_2 = values.quantile(0.025)
upper = values.quantile(0.975) 
mean_2 = values.mean()
risk_2 = (values < 0).mean() * 100
print ("Риски региона_2:", risk_2)
print("Средняя выручка региона_2:", mean_2)
print("2,5%-квантиль:", lower_2)

Риски региона_2: 6.2
Средняя выручка региона_2: 419919676.28175795
2,5%-квантиль: -115852609.16001143


In [44]:
print (mean_0)
print (lower_0)
print (risk_0)
print ( )
print (mean_1)
print (lower_1)
print (risk_1)
print ()
print (mean_2)
print (lower_2)
print (risk_2)

425368536.2814983
-102090094.83793654
6.0

518749628.38254464
129518062.69146843
0.3

419919676.28175795
-115852609.16001143
6.2


<div class="alert alert-info">
Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.
<br> 
    <br>
У региона0 и региона2 средний запас сырья больше, чем у региона1. Но среднеквадратичное отклонение больше на два порядка. То есть предположительный запас сырья у региона1 меньше, но предсказать это количество мы можем с большей точностью, чем для других регионов.
    <br>
    <br>
Если смотреть на среднюю выручку по регионам, то она выше у региона1, и квантиль тоже единственный неотрицательный. Риски однозначно самые низкие
    <br>
Поэтому однозначно надо выбирать регион1.
</div